In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,mapping,LineString, Point
import math
import tqdm
import sqlite3
from calculo_predial import *
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from Herramientas import *

In [ ]:
# Inputs

# Modelo IA
path_modelo_casas = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Ixtapan_Sal_casas.shp'
# Manzanas del municipio
path_manzanas     = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Manzana_Ixtapan_Sal.shp'
# Cruce de valores unitarios vs Manzana
#path_vu_and_mz    = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Valores_Unitarios_con_Manzanas_Naucalpan.shp'
# Valores unitarios
path_sql          ='Data/Valores_Unitarios_2022_v2'
# shape 15m 
path_info = '/home/hector/Documentos/Infis/Geo/Data/Shapes/15m/15m.shp'
# path de chinchetas
path_chinchetas = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/chinchetas_limpieza1/Centro_chinchetas_limpieza1.shp'
# ruta a dnue
path_dnue = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Dnue/DENUE_EDOMEX_2022.shp'
# Lista de afirmaciones en minisculas
afirmativo = np.array(['si','1',1,'yes','claro','obvio','of course','sip','sipi','neta','ok'])
# Municipio a procesar
municipio = 'Ixtapan_de_la_sal'

## Proceso 0

In [ ]:
# Importar valores de SQL

conexion = sqlite3.connect(path_sql)
df_area = pd.read_sql(f'''
SELECT Zona,Manzanas,COD,Tipo,Frente,Fondo,Area as Area_AH,[Val m2],
        Pseudo_clave_cat, Municipio
FROM area_homogenea	
WHERE Municipio = '{municipio}'
    ''', conexion)
df_banda = pd.read_sql(f'''
SELECT [Tipo calle], [Nombre Calle], [Zona], [Manzanas], COD,
       [Val m2], Pseudo_clave_cat, Municipio
FROM banda_valores
WHERE Municipio = '{municipio}'
''', conexion)

conexion.close()

In [ ]:
# Corregir formato del val m2 en ambas
df_area['Val m2'] = df_area['Val m2'].map(lambda x: corregir_formato(str(x)))
df_banda['Val m2'] = df_banda['Val m2'].map(lambda x: corregir_formato(str(x)))

In [ ]:
# Visualizar que tenemos el municipio adecuado
df_area['Municipio'].value_counts(), df_banda['Municipio'].value_counts()

In [ ]:
# Borramos duplicados
print('Shape inicial de Area_H ', df_area.shape)
print('Shape inicial de bandaV ', df_banda.shape)
df_area.drop_duplicates(subset=['Zona', 'Manzanas', 'COD', 'Tipo', 'Frente', 'Fondo', 'Area_AH',
       'Val m2', 'Pseudo_clave_cat'],
       inplace=True)
df_banda.drop_duplicates(subset=['Tipo calle', 'Nombre Calle', 'Zona', 'Manzanas', 'COD', 'Val m2',
       'Pseudo_clave_cat'], inplace=True)
print('Shape Final de Area_H ', df_area.shape)
print('Shape Final de bandaV ', df_banda.shape)

In [ ]:
# Cargamos el shape de manzanas para analisis
shape = gpd.read_file(path_manzanas)
shape=shape.to_crs("3857")
shape['key'] = shape['cve_cat'].map(lambda x: str(x)[:8])
shape['Lon'] = shape['geometry'].centroid.x
shape['Lat'] = shape['geometry'].centroid.y
shape.shape

In [ ]:
print(f'Shape de Area Homo= {df_area.shape}')
print(f'Shape de Banda    = {df_banda.shape}')
print(f'Shape de Manzanas = {shape.shape}')

In [ ]:
len(df_banda[~df_banda["Manzanas"].isin(df_area["Manzanas"])]["Manzanas"].unique())

In [ ]:
# Cruce area_h vs Mzs
cruce1 = df_area.merge(shape, 
                       left_on='Pseudo_clave_cat', right_on='key', 
                       how='outer', indicator=True, 
                       suffixes=('_AreaH', '_Manz'))
cruce1['Tipo_cruce1'] = cruce1['_merge'].map({'both':'AreaH_&_Manzana', 'left_only':'AreaH', 'right_only':'Manzanas' })
cruce1.drop(columns=['_merge'], inplace=True)
cruce1.shape

In [ ]:
# Cruce anterior vs BandaV
cruce1 = df_banda.merge(cruce1, 
                        left_on='Pseudo_clave_cat', right_on='key', 
                        how='outer', indicator=True, 
                        suffixes=('_BandaV', '_Manz'))
cruce1['Tipo_cruce2'] = cruce1['_merge'].map({'both':'BandaV_&_Manzana_&_AreaH', 'left_only':'BandaV', 'right_only':'Manzanas_&_AreaH' })
cruce1.drop(columns=['_merge'], inplace=True)
# Revisamos el tipo de variable
for col in ['Val m2_BandaV','Val m2_Manz']:
    cruce1[col] = cruce1[col].astype('float')
cruce1[['Val m2_BandaV','Val m2_Manz']].fillna(0, inplace=True)
# cruce1.sort_values('Val m2_Manz',inplace=True)
cruce1.shape

In [ ]:
# Borramos duplicados por clave catastral y geomety
print('Shape original --> ', cruce1.shape)
cruce1.drop_duplicates(subset=['cve_cat','geometry'],inplace=True)
print('Shape final    --> ', cruce1.shape)

In [ ]:
print(f'AreaH solo tiene  {len(df_area["Manzanas"].unique())} manzanas')
print(f'BandaV solo tiene {len(df_banda["Manzanas"].unique())} manzanas')
print(f'Total de manzanas diferentes que abarcan los valores unitarios {len(df_area["Manzanas"].unique()) + len(df_banda[~df_banda["Manzanas"].isin(df_area["Manzanas"])]["Manzanas"].unique())}')
print(f'Manzanas en el cruce BandaV: {len(cruce1["Manzanas_BandaV"].unique())}')
print(f'Manzanas en el cruce AreaH: {len(cruce1["Manzanas_Manz"].unique())}')

In [ ]:
# Identificamos los que no tienen valores
cruce1.loc[(cruce1['Val m2_BandaV'].fillna(0)==0) & (cruce1['Nombre Calle'].fillna(0)==0),['No_BandaV']] = 1
cruce1.loc[(cruce1['Val m2_Manz'].fillna(0)==0) & (cruce1['Tipo calle'].fillna('0')=='0'),'No_AreaH'] = 1


In [ ]:
# Interpolamos para rellenar faltantes
cruce1['Val_m2_AH_I'] = cruce1['Val m2_Manz'].interpolate(method='pad') 

In [ ]:
# Convertir a Geodataframe el cruce1
cruce1 = gpd.GeoDataFrame(cruce1, geometry='geometry').to_crs(3857)

## Proceso 1
Asociar un valor unitario a cada registro del modelo IA, para poder aproximar su valor catastral

In [ ]:
# Revisamos las columnas para llenar la sig celda
cruce1.columns

In [ ]:
# Definimos columnas utiles
# Valores unitarios para banda de valores
col_vu_bv = 'Val m2_BandaV'
# Valores unitarios para area homogenea
col_vu_ah = 'Val_m2_AH_I'

In [ ]:
# Revision de valores 0, no permitidos en ambos Val m2
cruce1[(cruce1[col_vu_ah].fillna(0)==0)&(cruce1[col_vu_bv].fillna(0)==0)].shape

In [ ]:
# Cargamos el GeoDataFrame del modelo de la IA
gdf_modelo_c = gpd.read_file(path_modelo_casas).to_crs(3857)
gdf_modelo_c.rename(columns={'area':'Area_Modelo'}, inplace=True)
print('Shape original -> ',gdf_modelo_c.shape)
gdf_modelo_c.drop_duplicates(subset='geometry', inplace=True)
print('Shape final -> ',gdf_modelo_c.shape)

In [ ]:
len(gdf_modelo_c.geometry.unique())

In [ ]:
# Intentamos borrar la clase carros si es que existe
existe_categoria = False
try:
    s = gdf_modelo_c.shape[0]
    gdf_modelo_c.drop(index=gdf_modelo_c[gdf_modelo_c['clase_dete']=='carros'].index, inplace=True)
    print('Se borraron: ',s-gdf_modelo_c.shape[0])
    # Columnas de categorias
    casas = ['casas', 'establecimiento','multivivienda']
    terrenos = ['area_verde','terreno_baldio',]
    existe_categoria = True
except:
    print('Revise que la columnas para borrar carros exista')
    print(gdf_modelo_c.columns)

In [ ]:
# Cruce de modelo IA vs Valores Unitarios & mz
cruce2 = gpd.sjoin(gdf_modelo_c,cruce1, how='left', lsuffix='_Cruce1', rsuffix='_ModeloIA')
#cruce2['Tipo_cruce1'] = cruce2['Tipo_cruce1'].astype(str)
#cruce2['Tipo_cruce2'] = cruce2['Tipo_cruce2'].astype(str)
print('Shape original = ', cruce2.shape)
cruce2.reset_index(drop=True,inplace=True)
cruce2.dropna(subset=['cve_cat','geometry'], inplace=True)
print('Shape final    = ', cruce2.shape) 

In [ ]:
# Volvemos a revisar que los val por m2 esten llenos
cruce2[(cruce2[col_vu_bv].fillna(0)==0)&(cruce2[col_vu_ah].fillna(0)==0)]

In [ ]:
# Obtener las medidas de los lados
cruce2['Medida_lx'] = cruce2['geometry'].map(lambda x: get_lados_xy(x)[0]) 
cruce2['Medida_ly'] = cruce2['geometry'].map(lambda x: get_lados_xy(x)[1]) 

In [ ]:
# Cargamos 15m

# Clave de municipio
cve_mun = '040'


gdf_15m = gpd.read_file(path_info).to_crs(3857)

# Filtramos
gdf_15m = gdf_15m[gdf_15m['CVE_MUN']==cve_mun] 
gdf_15m['geometry'].tail(3)

In [ ]:
# Cruzamos 15m con el cruce2 (cruce IA vs Manzanas)
# Cruce 
cruce3 = gpd.sjoin(cruce2, gdf_15m,
                   how='left',lsuffix='_cruce2',rsuffix='_15m')
cruce3.tail(3)

In [ ]:
if cruce3[cruce3['manz'].fillna('vacio')=='vacio'].shape[0] == 0:
    print('ok')
    drop_mz = ['Zona_Manz','Manzanas_Manz','CVE_MZA']
else:
    print('revisa las filas vacias y buscales una manzana')
    drop_mz = []

In [ ]:
# Borrar cols inecesarias
cols_drop = ['key','Pseudo_clave_cat_BandaV','Pseudo_clave_cat_Manz',
             'Municipio_BandaV','Municipio_Manz','mun','index__15m','CVE_MUN',
             'CVE_ENT','CVE_LOC'] + drop_mz
print('Shape original: ', cruce3.shape)
cruce3.drop(columns=cols_drop,inplace=True)
print('Shape final: ', cruce3.shape)

In [ ]:
cruce3['AMBITO'].value_counts()

In [ ]:
cruce3['TIPOMZA'].value_counts()

## Proceso 1.1
Cruzamos con valores de chinchetas

In [ ]:
gdf_chinchetas = gpd.read_file(path_chinchetas)
gdf_dnue = gpd.read_file(path_dnue)

In [ ]:
schema_crs={'set_ch':3857,'to_ch':3857,'set_dnue':6364,'to_dnue':3857}


In [ ]:
gdf_chinchetas = gdf_chinchetas.set_crs(schema_crs['set_ch'],allow_override=True)
gdf_dnue   = gdf_dnue.set_crs(schema_crs['set_dnue'],allow_override=True)
gdf_chinchetas.set_geometry('geometry',inplace=True)

In [ ]:
gdf_chinchetas = cruzar_chinchetas_vs_dnue(gdf_chinchetas, gdf_dnue,
                                           municipio='Ixtapan de la Sal',
                                           show_crs=True,
                                           show_results=True)

In [ ]:
# Hacemos cruce IA vs chinchetas
cruce4 = gpd.sjoin(cruce3, gdf_chinchetas, how='inner',
          lsuffix='IA', rsuffix='ch')
# Creamos diccionario de agrupacion
cols_agg = ['Clase','tipoCenCom', 'tipo_asent', 'nombre_act', 'clee','id']
dict_agg = {}
for k in cols_agg:
    dict_agg[k] = '|'.join
    
# Hacemos todo str menos la geometry para agrupar
for col in cols_agg:
    if col == 'geometry':
        print('omitio ', col)
        continue
    cruce4[col] = cruce4[col].astype(str)
        

cruce4 = cruce4.groupby('geometry', as_index=False,sort=False).agg(dict_agg)


In [ ]:
cruce4.columns

In [ ]:
# Asignamos lo encontrado al original modelo IA
for i in tqdm.tqdm(cruce4.index):
    cruce3.loc[cruce3['geometry'] == cruce4.loc[i,'geometry'], cols_agg] = cruce4.loc[i,cols_agg]


In [ ]:
from shapely.geometry import Point
c = pd.read_csv('/home/hector/Descargas/temporal/test_igecem_ch.csv',)

geometry = [eval(x) for x in c.geometry.str.replace('POINT ','Point').str.replace(' ',',')]
c = gpd.GeoDataFrame(c, geometry=geometry,crs=6364)
print(c.crs)
c.head()


In [ ]:
cruce4.shape

In [ ]:

cruce_f = gpd.sjoin(cruce4, c, how='left',
          rsuffix='IA',lsuffix='test')
cruce_f.shape

## Proceso 2
Hacer una identificacion geodesica de cada registro en el modelo IA para poder obtener factores necesarios para la aproximacion del valor catastral

In [ ]:
# Hacemos un cambio de variable para mejor visualizacion
gdf = cruce3
gdf.reset_index(drop=False, inplace=True)
gdf.rename(columns={'index':'Indice_gdf'}, inplace=True)
gdf = gdf.to_crs(3857)
gdf['Esquinero|Intermedio'] = 0
gdf.shape

In [ ]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del cruce2, cruce1, gdf_modelo_c
        print('Matamos las variables cruce2, cruce1, gdf_modelo_c, ya no hay vuelta atras')

In [ ]:
# Cargamos el shape de manzanas para analisis
shape = gpd.read_file(path_manzanas)
shape=shape.to_crs("3857")
shape["centroid"]=shape.centroid
shape.shape

In [ ]:
# Volvemos a revisar que los val por m2 esten llenos
gdf[(gdf[col_vu_bv].fillna(0)==0)&(gdf[col_vu_ah].fillna(0)==0)]

In [ ]:
# Buscamos por distancias los mas cercanos al perimetro de la manzana

nuevos = get_polygons_nearest_perimeter(falta_fp=gdf,
                                        shape=shape,
                                        col_cve_cat='cve_cat',
                                        distancia_max=4,
                                        col_id='Indice_gdf')

In [ ]:
se_rompio_arriba = False
if se_rompio_arriba:
    shape.geometry = shape.geometry.map(lambda y: y.convex_hull)
    # Codigo para revisar los multipoligonos
    # shape[shape.geometry.map(lambda y: True if isinstance(y,MultiPolygon) else False)]

In [ ]:
# etiquetamos los valores que son corner_polygons con el otro algoritmo
gdf.loc[gdf['Indice_gdf'].isin(np.unique(nuevos['Indice_gdf'].values)), 'Esquinero|Intermedio'] = 1

In [ ]:
# Revision de conteos nuevos en el gdf
gdf['Esquinero|Intermedio'].value_counts()

In [ ]:
# Borrar duplicados
print('Shape original ', gdf.shape)
gdf.drop_duplicates(subset=['geometry','cve_cat'],inplace=True)
print('Shape fianl ', gdf.shape)

In [ ]:
# Volvemos a revisar que los val por m2 esten llenos
gdf[(gdf[col_vu_bv].fillna(0)==0)&(gdf[col_vu_ah].fillna(0)==0)]

## Proceso 3
Ahora ya tenemos la info necesaria para el calculo de los factores lo que nos llevara al valor catastral

Todo este procedimiento a implementar es un seguimiento detallado del [manual catastral](!https://igecem.edomex.gob.mx/sites/igecem.edomex.gob.mx/files/files/ArchivosPDF/Servicios-catastrales/Manual%20Catastral%20del%20Estado%20de%20Mexico.pdf) que el IGECEM publica para el calculo del valor catastral. 

In [ ]:
datos = gdf
#datos.fillna(0,inplace=True)

In [ ]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del gdf
        print('Matamos las variables cgdf   , ya no hay vuelta atras')

In [ ]:
# borrar duplicados again
print('Shape original -> ',datos.shape)
datos.drop_duplicates(subset=['geometry','cve_cat'],inplace=True)
print('Shape nuevo    -> ',datos.shape)

In [ ]:
# Obtenemos las columnas de altura, nivel, y valores para borrarlas, ya que se hara de nuevo la clasificacion
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_otros = np.array([])
col_categoria = None
#datos.fillna(0, inplace=True)


for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    # elif col in list(vu_const['Clase']):
    #     cols_otros = np.append(cols_otros, col)
    elif col.lower().strip().startswith('clase_dete'):
        col_categoria = col
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_otros), col_categoria

In [ ]:
# Diccionarios utiles para clasificacion

dict_niveles = {
    'Niveles_Bajo':1,
    'Niveles_Medio':2,
    'Niveles_Alto':3,
    'Niveles_oficina':5,
    'Niveles_edificios':10
}

In [ ]:
# Renombramos en caso de ser necesario
renombres = {'Area_H':'Area_AH'}
datos.rename(columns=renombres,inplace=True)

In [ ]:
# Columnas a utilizar, pueden variar, por eso se definen aqui
col_area_modelo = 'Area_Modelo'
col_area_h = 'Area_AH'

In [ ]:
# Poner niveles 
for key,val in dict_niveles.items():
    print(key,val)
    datos[key] = val

In [ ]:
path_vu_construccion = './Data/Valores unitarios Construccion.xlsx'
df_construccion = pd.read_excel(path_vu_construccion)
df_construccion.tail()

In [ ]:
# Poner valores unitarios de construccion

cols_vu_cons = np.array([])
usar = 'baja'  # baja, media, alta


for tipo in datos['Tipo'].unique():
    # print(tipo)
    tipo = str(tipo).replace(' ','')
    if tipo == '0':
        tipo = 'H4'
    df_aux = df_construccion.loc[df_construccion['Codigo']==tipo]
    for clase, val in df_aux[['Clase','Valor m2']].values:
        if str(clase).endswith(usar):
            print(clase, val)
            datos.loc[(datos['Tipo']==tipo), clase] = val
            cols_vu_cons = np.append(cols_vu_cons,clase)
            del clase, val
        else:
            continue
    del df_aux

In [ ]:
# Obtenemos las columnas de altura, nivel, y valores
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_const = np.array([])
#datos.fillna(0, inplace=True)

for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    # elif col in list(vu_const['Clase']):
    #     cols_const = np.append(cols_const,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni),# len(cols_const)

In [ ]:
# Campos faltantes
datos['Frente_base'] = datos['Frente']
datos['Fondo_base']  = datos['Fondo']
datos['Area_base']   = datos[col_area_h]

if existe_categoria:
    datos.loc[datos[col_categoria].isin(casas),'grado_conservacion'] = datos.loc[datos[col_categoria].isin(casas)][col_area_h].map(lambda x:get_grado_conservacion())
else:
    datos['grado_conservacion'] = datos[col_area_h].map(lambda x:get_grado_conservacion())
datos['Area_inscrita']     = datos[col_area_modelo].map(lambda x: float(x)*random.uniform(0.65,1))
datos['Area_construccion'] = datos[col_area_modelo]*.9

In [ ]:
# Columnas a utilizar, pueden variar, por eso se definen aqui
col_esquinero = 'Esquinero|Intermedio'

In [ ]:
for col in ['Tipo_cruce1','Tipo_cruce2']:
    datos[col] = datos[col].astype(str)

In [ ]:
datos['Area_base']  = datos['Area_base'].str.replace(',','').astype(float)

In [ ]:
# Factores catastral del terreno

datos.fillna(0, inplace=True)
cols_factor_top = np.array([])

datos['Factor_posicion']      = datos[col_esquinero].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Interior' if x==0.5 else 'otro')
datos['factor_frente']        = datos.apply(lambda x: factor_frente(x['Medida_lx'], x['Terreno_Posicion']),axis=1)
datos['factor_fondo']         = datos.apply(lambda x: factor_fondo(x['Medida_ly'], x['Fondo_base'], x['Terreno_Posicion']), axis=1)
datos['Factor_Topografia']    = datos.apply(lambda x: round(random.uniform(0.9,1),5), axis=1)



datos['factor_irregularidad'] = datos.apply(lambda x: factor_irregularidad(x[col_area_modelo], x['Area_inscrita'], x['Terreno_Posicion']), axis=1)
datos['factor_area']          = datos.apply(lambda x: factor_area(x[col_area_modelo],x['Area_base'],x['Terreno_Posicion']), axis=1)
datos['factor_restriccion']   = datos.apply(lambda x: factor_restriccion(x[col_area_modelo],x[col_area_modelo]*0.8,x['Terreno_Posicion']), axis=1)

In [ ]:
# Revison de valores 0 en los factores de terreno y correccion

cols_ft = {'factor_frente':0.5,'factor_fondo':0.6,'factor_irregularidad':0.5,'factor_area':0.7,'factor_restriccion':0.5}

for col, val in cols_ft.items():
    s = datos[datos[col].fillna(0)==0].shape
    
    if s[0] != 0:
        datos.loc[datos[col].fillna(0)==0, col] = val
        print(col)
    print(s)
    
for col in cols_factor_top:
    s = datos[datos[col].fillna(0)==0].shape
    if s[0] != 0:
        print(col)
    print(s)

In [ ]:
# Factores catastral de la construcción

cols_factor_nvl = np.array([])

if existe_categoria:
    datos.loc[datos[col_categoria].isin(casas),'factor_grado_conservacion'] = datos[datos[col_categoria].isin(casas)]['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
    datos.loc[datos[col_categoria].isin(casas),'factor_edad']               = 0.6
else:
    datos['factor_grado_conservacion'] = datos['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
    datos['factor_edad']               = 0.6


for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    if existe_categoria:
        datos.loc[datos[col_categoria].isin(casas),nombre] = datos[datos[col_categoria].isin(casas)].apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)
    else:
        datos[nombre] = datos.apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)

In [ ]:
# Revision de valores 0 en los factores de construccion
if existe_categoria:
    print(datos[(datos[col_categoria].isin(casas))&(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
    print(datos[(datos[col_categoria].isin(casas))&(datos['factor_edad'].fillna(0)==0.0)].shape)
    
    for col in cols_factor_nvl:
        s = datos[(datos[col_categoria].isin(casas))&(datos[col].fillna(0)==0.0)].shape
        if s[0] != 0:
            print(col)
        print(s)
else:
    print(datos[(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
    print(datos[(datos['factor_edad'].fillna(0)==0.0)].shape)
    
    for col in cols_factor_nvl:
        s = datos[(datos[col].fillna(0)==0.0)].shape
        if s[0] != 0:
            print(col)
        print(s)


In [ ]:
# Conversion a flotantes de los valores que necesitamos

cols_using = [col_area_modelo,'factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
datos.fillna(0,inplace=True)
for col in cols_using:
    datos[col] = datos[col].astype(float)

![alt text](Data/Valor_cat_terreno.png) 

Formula para obtener el valor catastral del terreno

In [ ]:
# Valor catastral del terreno
print('Lista de valores unitarios para terreno \n ', cols_valor_uni)
index_use = input('Que valores quiere usar para el valor por metro cuadrado(separe cada indice con una coma, empezando en 0)?: ')
if index_use.lower() in ['todos', 'all']:
    cols_vu_terr = cols_valor_uni.copy()
else:
    cols_vu_terr = cols_valor_uni[[int(x) for x in index_use.split(',')]]
cols_vc_terr = np.array([])

for col_vu in cols_vu_terr:
    name = col_vu.split('_')[-1]
    name_col = 'Valor_catastral_terreno_'+name
    datos[name_col] = datos[col_area_modelo].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos['Factor_Topografia'].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
    cols_vc_terr = np.append(cols_vc_terr,name_col)

![alt text](Data/Valor_cat_const.png) <br>
 Formula para obtener el valor catastral de la construccion

In [ ]:
# Valor catastral de la construccion
cols_vc_const = np.array([])
for col_vu in cols_vu_cons:
    i = 0
    name = col_vu.split('_')[-1]
    for col_nv in cols_factor_nvl:
        name_col = 'Valor_catastral_construccion_'+name+'_'+str(i)
        cols_vc_const = np.append(cols_vc_const,name_col)
        datos[name_col] = datos[col_area_modelo].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
        i += 1

In [ ]:
# Revicion del total de columnas creadas
print(f'Cols construccion -> {len(cols_vc_const)} \nCols terreno      -> {len(cols_vc_terr)} \nShape Final       -> {datos.shape}')

In [ ]:
# Volvemos a revisar que los val por m2 esten llenos
datos[(datos[col_vu_bv].fillna(0)==0)&(datos[col_vu_ah].fillna(0)==0)]

#### Checkpoint opcional
 <br>Hasta este punto hemos aproximado tantos valores catastrales como el numero de columnas creadas arriba.
El siguiente proceso consume muchos recurso asi que se recomienda llevarlo a google colab

In [ ]:
nombre_file = 'New_casas_Naucalpan_proces2_v1'

In [ ]:
# Crear en CSV para no truncar el nombre de columnas
pd.DataFrame(datos).to_csv('Data/Completados/Modelo_Final/'+nombre_file+'.csv',
                           index=False,
                           encoding='utf8')

In [ ]:
# Crear el geopandas solo con las columnas utiles para reponerselas al csv
datos[['geometry', 'cve_cat','Indice_gdf']].to_file('Data/Completados/Modelo_Final/Recuperacion_geometry/'+nombre_file+'.shp',
                                         index=False)

#### Checkpoint opcional

In [ ]:
# Camnibio de variable otra vez
df = datos
df.reset_index(drop=False, inplace=True)
df.rename(columns={'index':'Indice_Modelo'}, inplace=True)
df.shape

In [ ]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del datos
        print('Matamos las variables datos   , ya no hay vuelta atras')

In [ ]:
# Obtenemos las columnas de interes
cols_terreno = np.array([])
cols_const = np.array([])
for col in df.columns:
    if col.lower().find('valor_catastral_construccion')>=0:
        cols_const = np.append(cols_const, col)
    elif col.lower().find('valor_catastral_terreno')>=0:
        cols_terreno = np.append(cols_terreno, col)
len(cols_terreno), len(cols_const)

In [ ]:
# Volvemos a revisar que los val por m2 esten llenos
df[(df[col_vu_bv].fillna(0)==0)&(df[col_vu_ah].fillna(0)==0)]

In [ ]:
df[cols_const].info()

In [ ]:
# Obtenemos los estadisticos de interes, esta parte es la mas pesada
df['Estadisticos_terreno'] = df.apply(lambda x: statistics_values(x[cols_terreno]), axis=1)
df[df['Estadisticos_terreno']=='Revisar'].shape

In [ ]:
# Obtenemos los estadisticos de interes, esta parte es la mas pesada
df['Estadisticos_construccion'] = df.apply(lambda x: statistics_values(x[cols_const]), axis=1)
df[df['Estadisticos_construccion']=='Revisar'].shape

In [ ]:
# Damos formatoa las cols Minimo, Maximo, Media, desviacion_estandar, largo, media_percentil, desviacion_std_percentiles, moda_percentiles
df = pd.concat([df,
                df['Estadisticos_terreno'].str.split('|', expand=True).rename(columns={0:   'Min_terreno',
                                                                                       1:   'Max_terreno',
                                                                                       2: 'Media_terreno',
                                                                                       3:   'STD_terreno',
                                                                                       4:   'Len_terreno',
                                                                                       5:'MediaP_terreno',
                                                                                       6:  'STDP_terreno',
                                                                                       7: 'ModaP_terreno'})], axis=1)

df = pd.concat([df,
                df['Estadisticos_construccion'].str.split('|', expand=True).rename(columns={0:   'Min_construccion',
                                                                                            1:   'Max_construccion',
                                                                                            2: 'Media_construccion',
                                                                                            3:   'STD_construccion',
                                                                                            4:   'Len_construccion',
                                                                                            5:'MediaP_construccion',
                                                                                            6:  'STDP_construccion',
                                                                                            7: 'ModaP_construccion'})], axis=1)

Calculo del intervalo de confianza suponiendo distribucion normal en los valores catastrales

In [ ]:
# Valor z con nivel de significancia al 95%
z = stats.norm.ppf(.95)
z

![alt text](./Data/Valor_cat_final.png)

Formula para obtener el valor catastral final 

![alt text](./Data/Intervalo_confianza_norm.png) Formula para intervalo de confianza

In [ ]:
cols_estadisticos = ['Media_terreno','STD_terreno','Len_terreno','Media_construccion','STD_construccion','Len_construccion']

for col in cols_estadisticos:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [ ]:
df.head(2)

In [ ]:
df['Media_terreno']#- (z*df['STD_terreno']/df['Len_terreno'])

In [ ]:
# Limite inferior (LI) y superior (LS) de cada valor catastral
df['LI_95%_vc_terreno'] = df['Media_terreno'].astype(float).fillna(0)- (z*df['STD_terreno'].astype(float).fillna(0)/df['Len_terreno'].astype(float).fillna(0))
df['LS_95%_vc_terreno'] = df['Media_terreno'].astype(float).fillna(0)+ (z*df['STD_terreno'].astype(float).fillna(0)/df['Len_terreno'].astype(float).fillna(0)) 

df['LI_95%_vc_construccion'] = df['Media_construccion'].astype(float).fillna(0)- (z*df['STD_construccion'].astype(float).fillna(0)/df['Len_construccion'].astype(float).fillna(0))
df['LS_95%_vc_construccion'] = df['Media_construccion'].astype(float).fillna(0)+ (z*df['STD_construccion'].astype(float).fillna(0)/df['Len_construccion'].astype(float).fillna(0) )

df['LS_95%_catastral_final'] = df['LS_95%_vc_terreno'].fillna(0) + df['LS_95%_vc_construccion'].fillna(0) 
df['LI_95%_catastral_final'] = df['LI_95%_vc_terreno'].fillna(0) + df['LI_95%_vc_construccion'].fillna(0) 

In [ ]:
df[[col_area_modelo,col_vu_ah, 'LI_95%_catastral_final','LS_95%_catastral_final']].head(3)

In [ ]:
cols_estadisticos = ['Media_terreno','STD_terreno','Len_terreno','Min_terreno','Max_terreno','MediaP_construccion','STDP_construccion','ModaP_construccion',
                     'Media_construccion','STD_construccion','Len_construccion','Min_construccion','Max_construccion','MediaP_construccion','STDP_construccion','ModaP_construccion',
                     'LI_95%_catastral_final','LS_95%_catastral_final']

for col in cols_estadisticos:
    try:
        if col.lower().startswith('min'):
            df[col] = df[col].str.replace('Revisar','0')
        df[col] = df[col].astype(float)
    except Exception as e:
        print(col)
        print(e)

In [ ]:
# Calculo de valor catastral para estadisticos
df['minVC'] = df['Min_terreno'].astype(float).fillna(0) + df['Min_construccion'].astype(float).fillna(0)
df['maxVC'] = df['Max_construccion'].astype(float).fillna(0) + df['Max_terreno'].astype(float).fillna(0)
df['mediaVC'] = df['Media_construccion'].astype(float).fillna(0) + df['Media_terreno'].astype(float).fillna(0)
df['PmediaVC'] = df['MediaP_construccion'].astype(float).fillna(0) + df['MediaP_terreno'].astype(float).fillna(0)

In [ ]:
# Solo cols de interes a
df[np.append(cols_estadisticos,['Indice_gdf','geometry','cve_cat'])]

#### En caso de haber hecho checkpoint

In [ ]:
# En caso de haber hecho checkpoint
path_gdf =  'Data/Completados/Modelo_Final/Recuperacion_geometry/'+nombre_file+'.shp'
gdf = gpd.read_file(path_gdf)
gdf.rename(columns={'Indice_gdf':'Indice_gdf1'},inplace=True)

final = gpd.GeoDataFrame(pd.concat([gdf[['geometry', 'cve_cat','Indice_gdf1']] , df[cols_new]], axis=1), geometry='geometry')
final.tail(2)

gdf.shape, final.shape

In [ ]:
# Revision de empatamiento 
final[final['Indice_gdf']!=final['Indice_gdf1']]

In [ ]:
# Exportar el GeoDataFrame
final.to_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/Completados/Modelo_Final/Solo_VC_Final/NewCasas_Valor_Catastral_Naucalpan_Final_v1.shp',
                                                  index=False)

#### En caso de no haber hecho checkpoint

In [ ]:
# Calculo del predial
cols_calc_predial = ['minVC','maxVC','mediaVC','LI_95%_catastral_final','LS_95%_catastral_final','PmediaVC']
cols_predial = []
df_predial = pd.read_csv('./Data/Predial.csv',)
df_predial['FACTOR'] = df_predial['FACTOR'].str.replace(',','.')
for col in df_predial:
    df_predial[col] = df_predial[col].astype(float)


for col in cols_calc_predial:
    name_col = 'Predial'+col
    df[name_col] = df[col].map(lambda x: calculo_predial(x,df_predial))
    cols_predial.append(name_col)

In [ ]:
# Ya terminamos solo exportamos el geodataframe
ruta_and_name_save = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Ixtapan_completo_VC_v2.shp'
if input('usar cols solo necesarias') in afirmativo:
    cols_out = np.append(cols_predial,['cve_cat','Indice_gdf','geometry']+cols_agg)
else:
    cols_out = df.columns
    
df[cols_out].to_file(ruta_and_name_save,  index=False)

In [ ]:
df['comparacion_min_max'] = df['PredialminVC'] == df['PredialmaxVC']
df['comparacion_min_max'].value_counts()

In [ ]:
df[np.append(cols_predial,['cve_cat','Indice_gdf','geometry'])]